In [2]:
from torch import nn,optim
import torch
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt
import numpy as np
torch.manual_seed(1)

In [4]:
class Data(Dataset):
    def __init__(self,train=True):
        self.x = torch.arange(-3.0,3.0,0.1).view(-1,1)
        self.f = -3 * self.x + 1
        self.y = self.f - 0.3 * torch.randn(self.x.size())
        self.len = self.x.size()[0]
        
        #outliers
        if train == True:
            self.y[0] = 0
            self.y[50:55] = 20
        else:
            pass
    def __getitem(self,index):
        return self.x[index],self.y[index]
    def __len__(self):
        return self.len

In [5]:
train_data = Data()
val_data = Data(train=False)

In [6]:
class linear_regression(nn.Module):
    def __init__(self,inp_size,out_size):
        super(linear_regression,self).__init__()
        self.linear = nn.Linear(inp_size,out_size,bais=True)
    
    def forward(self,x):
        yhat = self.linear(x)
        return yhat

In [7]:
criterion  = nn.MSELoss()
trainloader = DataLoader(dataset=train_data,batch_size=1)

In [8]:
learning_rates = [0.0001,0.001,0.01,0.1]
train_error = torch.zeros(len(learning_rates))
validation_error = torch.zeros(len(learning_rates))

MODELS=[]

In [ ]:
def train_model_with_lr(iter,lr_list):
    #go through different learning rate
    for i,lr in enumerate(lr_list):
        model = linear_regression(1,1)
        optimizer = optim.SGD(model.parameters(),lr=lr)
        for epoch in range(iter):
            for x,y in trainloader:
                yhat = model.forward(x)
                loss = criterion(yhat,y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
        #train loss from each lr        
        Yhat = model(train_data.x)
        train_loss = criterion(Yhat,train_data.y)
        train_error[i] = train_loss.item()
        
        #validation data
        Yhat = model(val_data.x)
        val_loss = criterion(Yhat,val_data.y)
        validation_error[i] = val_loss.item()
        MODELS.append(model)
train_model_with_lr(5,learning_rates)
                